# Spectral Signal

## Implementation

In [1]:
import numpy as np
from contextlib import contextmanager
from copy import deepcopy
from collections import OrderedDict
from pandas import Series

import colour
from colour import (
    CaseInsensitiveMapping,
    CubicSplineInterpolator,
    Extrapolator,
    LinearInterpolator,
    PchipInterpolator,
    SpragueInterpolator,
    as_numeric,
    is_numeric,
    tsplit,
    tstack,
    warning)


class NullInterpolator(object):
    """

    Parameters
    ----------
    x : ndarray
        Independent :math:`x` variable values corresponding with :math:`y`
        variable.
    y : ndarray
        Dependent and already known :math:`y` variable values to
        interpolate.

    Methods
    -------
    __call__


    Examples
    --------
    """

    def __init__(self,
                 x=None,
                 y=None,
                 absolute_tolerance=10e-7,
                 relative_tolerance=10e-7):
        self.__x = None
        self.x = x
        self.__y = None
        self.y = y
        self.__absolute_tolerance = None
        self.absolute_tolerance = absolute_tolerance
        self.__relative_tolerance = None
        self.relative_tolerance = relative_tolerance


        self.__validate_dimensions()

    @property
    def x(self):
        """
        Property for **self.__x** private attribute.

        Returns
        -------
        array_like
            self.__x
        """

        return self.__x

    @x.setter
    def x(self, value):
        """
        Setter for **self.__x** private attribute.

        Parameters
        ----------
        value : array_like
            Attribute value.
        """

        if value is not None:
            value = np.atleast_1d(value).astype(np.float_)

            assert value.ndim == 1, (
                '"x" independent variable must have exactly one dimension!')

        self.__x = value

    @property
    def y(self):
        """
        Property for **self.__y** private attribute.

        Returns
        -------
        array_like
            self.__y
        """

        return self.__y

    @y.setter
    def y(self, value):
        """
        Setter for **self.__y** private attribute.

        Parameters
        ----------
        value : array_like
            Attribute value.
        """

        if value is not None:
            value = np.atleast_1d(value).astype(np.float_)

            assert value.ndim == 1, (
                '"y" dependent variable must have exactly one dimension!')

        self.__y = value

    @property
    def relative_tolerance(self):
        """
        Property for **self.__relative_tolerance** private attribute.

        Returns
        -------
        numeric
            self.__relative_tolerance
        """

        return self.__relative_tolerance

    @relative_tolerance.setter
    def relative_tolerance(self, value):
        """
        Setter for **self.__relative_tolerance** private attribute.

        Parameters
        ----------
        value : numeric
            Attribute value.
        """

        if value is not None:
            assert is_numeric(value), (
                '"relative_tolerance" variable must be a "numeric"!')

        self.__relative_tolerance = value

    @property
    def absolute_tolerance(self):
        """
        Property for **self.__absolute_tolerance** private attribute.

        Returns
        -------
        numeric
            self.__absolute_tolerance
        """

        return self.__absolute_tolerance

    @absolute_tolerance.setter
    def absolute_tolerance(self, value):
        """
        Setter for **self.__absolute_tolerance** private attribute.

        Parameters
        ----------
        value : numeric
            Attribute value.
        """

        if value is not None:
            assert is_numeric(value), (
                '"absolute_tolerance" variable must be a "numeric"!')

        self.__absolute_tolerance = value

    def __call__(self, x):
        """
        Evaluates the interpolatior at given point(s).


        Parameters
        ----------
        x : numeric or array_like
            Point(s) to evaluate the interpolant at.

        Returns
        -------
        float or ndarray
            Interpolated value(s).
        """

        x = np.atleast_1d(x).astype(np.float_)

        xi = as_numeric(self.__evaluate(x))

        return xi

    def __evaluate(self, x):
        """
        Performs the interpolator evaluation at given points.

        Parameters
        ----------
        x : ndarray
            Points to evaluate the interpolant at.

        Returns
        -------
        ndarray
            Interpolated points values.
        """

        self.__validate_dimensions()
        self.__validate_interpolation_range(x)

        index = nearest_index(self.__x, x)
        values = self.__y[index]
        values[~np.isclose(self.__x[index],
                           x,
                           rtol=self.__absolute_tolerance,
                           atol=self.__relative_tolerance)] = 0

        return values

    def __validate_dimensions(self):
        """
        Validates variables dimensions to be the same.
        """

        if len(self.__x) != len(self.__y):
            raise ValueError(
                ('"x" independent and "y" dependent variables have different '
                 'dimensions: "{0}", "{1}"').format(len(self.__x),
                                                    len(self.__y)))

    def __validate_interpolation_range(self, x):
        """
        Validates given point to be in interpolation range.
        """

        below_interpolation_range = x < self.__x[0]
        above_interpolation_range = x > self.__x[-1]

        if below_interpolation_range.any():
            raise ValueError('"{0}" is below interpolation range.'.format(x))

        if above_interpolation_range.any():
            raise ValueError('"{0}" is above interpolation range.'.format(x))


INTERPOLATORS = CaseInsensitiveMapping({
    'Cubic Spline': CubicSplineInterpolator,
    'Linear': LinearInterpolator,
    'Null' : NullInterpolator,
    'Pchip': PchipInterpolator,
    'Sprague': SpragueInterpolator})


def nearest_index(a, b):
    index = np.searchsorted(a, b)

    return np.where(np.abs(b - a[index-1]) < np.fabs(b - a[index]),
                   index - 1,
                   index)

def nearest(a, b):
    a = np.asarray(a)

    return a[nearest_index(a, b)]


@contextmanager
def ndarray_write(a):
    a.setflags(write=True)

    yield a

    a.setflags(write=False)


def fill_nan(a, method='Interpolation'):
    mask = np.isnan(a)

    if method.lower() == 'interpolation':
        a[mask] = np.interp(
            np.flatnonzero(mask),
            np.flatnonzero(~mask),
            a[~mask])
    elif method.lower() == 'zeros':
        a[mask] = 0

    return a


def is_pandas_installed():
    try:
        import pandas

        return True
    except ImportError:
        return False


def unpack_data(data=None, domain=None):
    domain_f, range_f, name_f = None, None, None
    if isinstance(data, Signal):
        domain_f = data.domain
        range_f = data.range
    if (isinstance(data, tuple) or
            isinstance(data, list) or
            isinstance(data, np.ndarray)):
        data = np.asarray(data)
        if data.ndim == 1:
            range_f = data
        elif data.ndim == 2:
            domain_f, range_f = tsplit(data)
        else:
            raise ValueError('"data" must be a 1d or 2d array-like variable!')
    elif (isinstance(data, dict) or
              isinstance(data, OrderedDict)):
        domain_f, range_f = tsplit(sorted(data.items()))
    elif is_pandas_installed():
        if isinstance(data, Series):
            domain_f = data.index.values
            range_f = data.values
            name_f = data.name

    domain_f = domain_f if domain is None else domain

    return domain_f, range_f, name_f


class Signal(object):
    def __init__(self,
                 data=None,
                 domain=None,
                 interpolation_method=None,
                 interpolation_options=None,
                 extrapolation_method=None,
                 extrapolation_options=None,
                 name=None):

        self.__domain = None
        self.__range = None
        self.__interpolation_method = 'Pchip'
        self.__interpolation_options = {}
        self.__extrapolation_method = 'Constant'
        self.__extrapolation_options = {'left': np.nan, 'right': np.nan}
        self.__name = '{0} ({1})'.format(self.__class__.__name__, id(self))

        domain_f, range_f, name_f = unpack_data(data, domain)
        name_f = name_f if name is None else name

        self.domain = domain_f
        self.range = range_f
        self.interpolation_method = interpolation_method
        self.interpolation_options = interpolation_options
        self.extrapolation_method = extrapolation_method
        self.extrapolation_options = extrapolation_options
        self.name = name_f

        self.__create_function()

    @property
    def range(self):
        return self.__range

    @range.setter
    def range(self, value):
        if value is not None:
            if not np.all(np.isfinite(value)):
                warning('"range" variable is not finite, '
                        'unpredictable results may occur!\n{0}'.format(value))

            value = np.asarray(value)

            if self.__domain is not None:
                assert value.size == self.__domain.size, (
                    '"domain" and "range" variables must have same size!')

            value.setflags(write=False)
            self.__range = value
            self.__create_function()

    @property
    def domain(self):
        return self.__domain

    @domain.setter
    def domain(self, value):
        if value is not None:
            if not np.all(np.isfinite(value)):
                warning('"domain" variable is not finite, '
                        'unpredictable results may occur!\n{0}'.format(value))

            value = np.asarray(value)

            if self.__range is not None:
                assert value.size == self.__range.size, (
                    '"domain" and "range" variables must have same size!')

            value.setflags(write=False)
            self.__domain = value
            self.__create_function()

    @property
    def interpolation_method(self):
        return self.__interpolation_method

    @interpolation_method.setter
    def interpolation_method(self, value):
        if value is not None:
            assert type(value) in (str, unicode), (  # noqa
                ('"{0}" attribute: "{1}" type is not '
                 '"str" or "unicode"!').format('interpolation_method', value))

            assert value in INTERPOLATORS, (
                ('"{0}" attribute: "{1}" interpolation method is not defined! '
                 'Available methods: "{2}".').format(
                    'interpolation_method',
                    value,
                    sorted(INTERPOLATORS.keys())))

            self.__interpolation_method = value
            self.__create_function()

    @property
    def interpolation_options(self):
        return self.__interpolation_options

    @interpolation_options.setter
    def interpolation_options(self, value):
        if value is not None:
            assert type(value) in (dict, OrderedDict), (
                ('"{0}" attribute: "{1}" type is not '
                 '"dict" or "OrderedDict"!').format(
                    'interpolation_options', value))

            self.__interpolation_options = value
            self.__create_function()

    @property
    def extrapolation_method(self):
        return self.__extrapolation_method

    @extrapolation_method.setter
    def extrapolation_method(self, value):
        if value is not None:
            assert type(value) in (str, unicode), (  # noqa
                ('"{0}" attribute: "{1}" type is not '
                 '"str" or "unicode"!').format('extrapolation_method', value))

            assert value in ('Constant', 'Linear'), (
                ('"{0}" attribute: "{1}" extrapolation method is not defined! '
                 'Available methods: "[\'Constant\', \'Linear\']".').format(
                    'interpolation_method', value))

            self.__extrapolation_method = value
            self.__create_function()

    @property
    def extrapolation_options(self):
        return self.__extrapolation_options

    @extrapolation_options.setter
    def extrapolation_options(self, value):
        if value is not None:
            assert type(value) in (dict, OrderedDict), (
                ('"{0}" attribute: "{1}" type is not '
                 '"dict" or "OrderedDict"!').format(
                    'extrapolation_options',value))

            self.__extrapolation_options = value
            self.__create_function()

    @property
    def name(self):
        return self.__name

    @name.setter
    def name(self, value):
        if value is not None:
            assert type(value) in (str, unicode), (  # noqa
                ('"{0}" attribute: "{1}" type is not '
                 '"str" or "unicode"!').format('name', value))
            self.__name = value

    @property
    def function(self):
        return self.__function

    @function.setter
    def function(self, value):
        raise AttributeError(
            '"{0}" attribute is read only!'.format('function'))

    def __create_function(self):
        if self.__domain is not None and self.__range is not None:
            self.__function = Extrapolator(
                INTERPOLATORS[self.__interpolation_method](
                    self.__domain, self.__range,
                    **self.__interpolation_options),
                method=self.__extrapolation_method,
                **self.__extrapolation_options)
        else:
            def __undefined_signal_interpolator_function(*args, **kwargs):
                raise RuntimeError(
                    'Underlying signal interpolator function does not exists, '
                    'please ensure you defined both '
                    '"domain" and "range" variables!')

            self.__function = __undefined_signal_interpolator_function

    def __getitem__(self, x):
        if type(x) is slice:
            return self.__range[x]
        else:
            return self.__function(x)

    def __setitem__(self, x, value):
        if type(x) is slice:
            with ndarray_write(self.__range):
                self.__range[x] = value
        else:
            with ndarray_write(self.__domain), ndarray_write(self.__range):
                x = np.atleast_1d(x)
                value = np.atleast_1d(value)

                # Matching index, replacing existing `self.range`.
                self.__range[np.in1d(self.__domain, x)] = value

                # Non matching index, inserting into existing `self.domain` and
                # `self.range`.
                x = x[~np.in1d(x, self.__domain)]
                indexes = np.searchsorted(self.__domain, x)

                self.__domain = np.insert(self.__domain, indexes, x)
                self.__range = np.insert(self.__range, indexes, value)

        self.__create_function

    def __iadd__(self, x):
        if isinstance(x, self.__class__):
            x = self.__function(x.domain)

        with ndarray_write(self.__range):
            self.range += x

        return self

    def __add__(self, x):
        copy = self.copy()
        copy += x

        return copy

    def __isub__(self, x):
        if isinstance(x, self.__class__):
            x = self.__function(x.domain)

        with ndarray_write(self.__range):
            self.range -= x

        return self

    def __sub__(self, x):
        copy = self.copy()
        copy -= x

        return copy

    def __imul__(self, x):
        if isinstance(x, self.__class__):
            x = self.__function(x.domain)

        with ndarray_write(self.__range):
            self.range *= x

        return self

    def __mul__(self, x):
        copy = self.copy()
        copy *= x

        return copy

    def __idiv__(self, x):
        if isinstance(x, self.__class__):
            x = self.__function(x.domain)

        with ndarray_write(self.__range):
            self.range /= x

        return self

    def __div__(self, x):
        copy = self.copy()
        copy /= x

        return copy

    __itruediv__ = __idiv__
    __truediv__ = __div__

    def copy(self):
        return deepcopy(self)

    def fill_nan(self, method='Interpolation'):
        with ndarray_write(self.__domain), ndarray_write(self.__range):
            self.__domain = fill_nan(self.__domain, method)
            self.__range = fill_nan(self.__range, method)
            self.__create_function()

    def uncertainty(self, x):
        n = nearest(self.__domain, x)

        return np.abs(x - n)

## Empty Object Initialisation

In [2]:
cs1 = Signal()

print('1) cs1[0]')
try:
    print(cs1[0])
except RuntimeError as error:
    print(error)

print('\n')

domain = np.arange(0, 1000, 100)
cs1 = Signal(domain=domain)
print('2) cs1[0]')
try:
    print(cs1[0])
except RuntimeError as error:
    print(error)

print('\n')

range = np.linspace(1, 10, domain.size)
cs1 = Signal(range, domain)
print('3) cs1[0]')
print(cs1[0])

print('\n')

print('4) cs1 = Signal(range, [])')
try:
    cs1 = Signal(range, [])
except AssertionError as error:
    print(error)

1) cs1[0]
Underlying signal interpolator function does not exists, please ensure you defined both "domain" and "range" variables!


2) cs1[0]
Underlying signal interpolator function does not exists, please ensure you defined both "domain" and "range" variables!


3) cs1[0]
1.0


4) cs1 = Signal(range, [])
"domain" and "range" variables must have same size!


## Object Initialisation

In [3]:
domain = np.arange(0, 1000, 100)
domain_a = np.linspace(0, 1, 10)
range = np.linspace(1, 10, domain.size)

data = zip(domain, range)

print('1) cs1 = Signal(range, domain)')
cs1 = Signal(range, domain)

print(cs1.name)
print(cs1.range)
print(cs1.domain)

print('\n')

print('2) cs1 = Signal(data)')
cs1 = Signal(data)

print(cs1.range)
print(cs1.domain)

print('\n')

print('3) cs1 = Signal(data, domain_a)')
cs1 = Signal(data, domain_a)

print(cs1.range)
print(cs1.domain)

print('\n')

print('4) cs1 = Signal(Signal(data))')
cs1 = Signal(Signal(data))

print(cs1.range)
print(cs1.domain)

print('\n')

print('5) cs1 = Signal(Series(range, domain))')
cs1 = Signal(Series(range, domain))

print(cs1.range)
print(cs1.domain)

print('\n')

print('6) cs1 = Signal(Series(range, domain, name="D65"))')
cs1 = Signal(Series(range, domain, name="D65"))

print(cs1.name)
print(cs1.range)
print(cs1.domain)

print('\n')

1) cs1 = Signal(range, domain)
Signal (140410287189584)
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[  0 100 200 300 400 500 600 700 800 900]


2) cs1 = Signal(data)
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[   0.  100.  200.  300.  400.  500.  600.  700.  800.  900.]


3) cs1 = Signal(data, domain_a)
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[ 0.          0.11111111  0.22222222  0.33333333  0.44444444  0.55555556
  0.66666667  0.77777778  0.88888889  1.        ]


4) cs1 = Signal(Signal(data))
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[   0.  100.  200.  300.  400.  500.  600.  700.  800.  900.]


5) cs1 = Signal(Series(range, domain))
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[  0 100 200 300 400 500 600 700 800 900]


6) cs1 = Signal(Series(range, domain, name="D65"))
D65
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[  0 100 200 300 400 500 600 700 800 900]




## Copy Operations

In [4]:
domain = np.arange(0, 1000, 100)
range = np.linspace(1, 10, domain.size)

cs1 = Signal(range, domain)

print('1) id(cs1)')
print(id(cs1))
print(cs1.function)

print('\n')

cs2 = cs1.copy()
print('2) id(cs2)')
print(id(cs2))
print(cs2.function)

1) id(cs1)
140411008644240


2) id(cs2)
140411008731408


## Item Operations

In [5]:
domain = np.arange(0, 1000, 100)
range = np.linspace(1, 10, domain.size)

cs1 = Signal(range, domain)

print('1) cs1')
print(cs1.range)
print(cs1.domain)

print('\n')

print('2) cs1[150.25]')
print(cs1[150.25])

print('\n')

print('3) cs1[np.linspace(100, 400, 10)]')
print(cs1[np.linspace(100, 400, 10)])

print('\n')

print('4) cs1[0:3]')
print(cs1[0:3])

print('\n')

print('5) cs1[10] = np.pi')
cs1[10] = np.pi
print(cs1.range)
print(cs1.domain)

print('\n')

print('6) cs1[(200, 300)] = np.pi')
cs1[(200, 300)] = np.pi
print(cs1.range)
print(cs1.domain)

print('\n')

print('7) cs1[(0, 850)] = np.pi')
cs1[(0, 850)] = np.pi
print(cs1.range)
print(cs1.domain)

print('\n')

print('8) cs1[0:9] = np.pi')
cs1[0:9] = np.pi
print(cs1.range)
print(cs1.domain)

1) cs1
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[  0 100 200 300 400 500 600 700 800 900]


2) cs1[150.25]
2.5025


3) cs1[np.linspace(100, 400, 10)]
[ 2.          2.33333333  2.66666667  3.          3.33333333  3.66666667
  4.          4.33333333  4.66666667  5.        ]


4) cs1[0:3]
[ 1.  2.  3.]


5) cs1[10] = np.pi
[  1.           3.14159265   2.           3.           4.           5.           6.
   7.           8.           9.          10.        ]
[  0  10 100 200 300 400 500 600 700 800 900]


6) cs1[(200, 300)] = np.pi
[  1.           3.14159265   2.           3.14159265   3.14159265   5.           6.
   7.           8.           9.          10.        ]
[  0  10 100 200 300 400 500 600 700 800 900]


7) cs1[(0, 850)] = np.pi
[  3.14159265   3.14159265   2.           3.14159265   3.14159265   5.           6.
   7.           8.           9.           3.14159265  10.        ]
[  0  10 100 200 300 400 500 600 700 800 850 900]


8) cs1[0:9] = np.pi
[  3.14159265   3.14

## Null Interpolator

In [6]:
domain = np.arange(0, 1000, 100)
range = np.linspace(1, 10, domain.size)

cs1 = Signal(range, domain, interpolation_method='Null')

print('1) cs1')
print(cs1.range)
print(cs1.domain)

print('\n')

print('2) cs1[100]')
print(cs1[100])

print('\n')

print('3) cs1[100.1, 500]')
print(cs1[100.1, 500])

print('\n')

print('4) cs1[100.0000001]')
print(cs1[100.0000001])

1) cs1
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[  0 100 200 300 400 500 600 700 800 900]


2) cs1[100]
2.0


3) cs1[100.1, 500]
[ 0.  6.]


4) cs1[100.0000001]
2.0


## Uncertainty

In [7]:
domain = np.arange(0, 1000, 100)
range = np.linspace(1, 10, domain.size)

cs1 = Signal(range, domain)

print('1) cs1')
print(cs1.range)
print(cs1.domain)

print('\n')

print('2) cs1.uncertainty((0.1, 150, 200))')
print(cs1.uncertainty((0.1, 150, 200)))

1) cs1
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[  0 100 200 300 400 500 600 700 800 900]


2) cs1.uncertainty((0.1, 150, 200))
[  0.1  50.    0. ]


## Arithmetical Operations with Matching Index

In [8]:
domain = np.arange(0, 1000, 100)
range = np.linspace(1, 10, domain.size)

cs1 = Signal(range, domain)
cs2 = Signal(range, domain)

print('1) cs1')
print(cs1.range)
print(cs1.domain)

print('2) cs2')
print(cs2.range)
print(cs2.domain)

print('\n')

print('3) cs1 += cs2')
cs1 += cs2
print(cs1.range)

print('\n')

print('4) cs1 += 1')
cs1 += 1
print(cs1.range)

print('\n')

print('5) cs1 -= np.ones(domain.size)')
cs1 -= np.ones(domain.size)
print(cs1.range)

print('\n')

print('6) cs2 = cs1 + cs1')
cs2 = cs1 + cs1
print(cs2.range)

print('\n')

1) cs1
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[  0 100 200 300 400 500 600 700 800 900]
2) cs2
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[  0 100 200 300 400 500 600 700 800 900]


3) cs1 += cs2
[  2.   4.   6.   8.  10.  12.  14.  16.  18.  20.]


4) cs1 += 1
[  3.   5.   7.   9.  11.  13.  15.  17.  19.  21.]


5) cs1 -= np.ones(domain.size)
[  2.   4.   6.   8.  10.  12.  14.  16.  18.  20.]


6) cs2 = cs1 + cs1
[  4.   8.  12.  16.  20.  24.  28.  32.  36.  40.]




## Arithmetical Operations with Mismatching Index

In [9]:
domain = np.arange(0, 1000, 100)
range = np.linspace(1, 10, domain.size)

cs1 = Signal(range, domain)
cs2 = Signal(range, domain + 400)

print('1) cs1')
print(cs1.range)
print(cs1.domain)

print('2) cs2')
print(cs2.range)
print(cs2.domain)

print('\n')

print('3) cs1 += cs2')
cs1 += cs2
print(cs1.range)

print('\n')

print('4) cs1 += 1')
cs1 += 1
print(cs1.range)

print('\n')

print('5) cs1 -= np.ones(domain.size)')
cs1 -= np.ones(domain.size)
print(cs1.range)

print('\n')

print('6) cs2 = cs1 + cs1')
cs2 = cs1 + cs1
print(cs2.range)

print('\n')

print('7) Cubic Spline interpolation fails with Nan(s) values.')
cs1.interpolation_method = 'Cubic Spline'
cs2 = cs1 + cs1
print(cs2.range)

print('\n')

print('8) Cubic Spline interpolation with filled Nan(s) values.')
cs1.fill_nan()
cs2 = cs1 + cs1
print(cs2.range)

1) cs1
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[  0 100 200 300 400 500 600 700 800 900]
2) cs2
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
[ 400  500  600  700  800  900 1000 1100 1200 1300]


3) cs1 += cs2
[  6.   8.  10.  12.  14.  16.  nan  nan  nan  nan]


4) cs1 += 1
[  7.   9.  11.  13.  15.  17.  nan  nan  nan  nan]


5) cs1 -= np.ones(domain.size)
[  6.   8.  10.  12.  14.  16.  nan  nan  nan  nan]


6) cs2 = cs1 + cs1
[ 12.  16.  20.  24.  28.  nan  nan  nan  nan  nan]


7) Cubic Spline interpolation fails with Nan(s) values.
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]


8) Cubic Spline interpolation with filled Nan(s) values.
[ 12.  16.  20.  24.  28.  32.  32.  32.  32.  32.]


/colour-science/colour/colour/utilities/verbose.py:125: UserWarning: "range" variable is not finite, unpredictable results may occur!
[  6.   8.  10.  12.  14.  16.  nan  nan  nan  nan]
  warn(*args, **kwargs)
/colour-science/colour/colour/utilities/verbose.py:125: UserWarning: "range" variable is not finite, unpredictable results may occur!
[  7.   9.  11.  13.  15.  17.  nan  nan  nan  nan]
  warn(*args, **kwargs)
/colour-science/colour/colour/utilities/verbose.py:125: UserWarning: "range" variable is not finite, unpredictable results may occur!
[ 12.  16.  20.  24.  28.  nan  nan  nan  nan  nan]
  warn(*args, **kwargs)
/colour-science/colour/colour/utilities/verbose.py:125: UserWarning: "range" variable is not finite, unpredictable results may occur!
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
  warn(*args, **kwargs)


## Spectral Power Distribution

In [10]:
class Spectrum(Signal):
    def __init__(self, *args, **kwargs):
        super(Spectrum, self).__init__(*args, **kwargs)